<a href="https://colab.research.google.com/github/selinaxin87/daily-briefings-py/blob/main/Project_Investigation_(Fall_2022).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Description





This app fetches pricing information for a given game across all available regions on STEAM, and displays the location with the lowest price from which the user may obtain the game they want.







Specifically, we will use the  AlphaVantage API's ["Currency Exchange Rate"](https://www.alphavantage.co/documentation/#currency-exchange) endpoint to **request real time exchange rates** for a given currency. 
>NOTE: We will compare prices under all [available currencies](https://partner.steamgames.com/doc/store/pricing/currencies) for a given game.

# Functionality Overview




We ask the user to input the app ID of a selected game. The corresponding app ID of the game would be provided within a separate link. 

We make a request for the prices of the selected games in all available STEAM regions, then convert the resulting prices into USD. 

Finally, we will display the lowest of the converted price in USD and make recommendation to the user on which regional STEAM store to purchase the selected game. 




# Information Requirements

### Information Inputs

  + Selected app ID
  + API response

### Information Outputs

  + API request
  + Printed lowest 

# Setup / Configuration

## API Credentials


To run the app, you first need to first obtain an [AlphaVantage API Key](https://www.alphavantage.co/support/#api-key). A normal key should be fine, but alternatively you can use one of the prof's "premium" keys. Once obtained, you will be asked to securely input this API Key via the setup cell below:

In [ ]:
#
# SETUP CELL (SECURE HANDLING OF SECRET CREDENTIALS)
#
# Run this cell to ask the user to securely input the API Key.
# Since the API Key is a secret credential like a password, 
# ... we want to avoid hard-coding or exposing it.
#

from getpass import getpass

API_KEY = getpass("Please input your AlphaVantage API Key: ")

Please input your AlphaVantage API Key: ··········


##Downloading Country Codes (CSV file)

In [ ]:
import os

url = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
filepath = "all.csv"

if not os.path.isfile(filepath):
    print("DOWNLOADING", filepath)
    # FYI: this wget command is a terminal command, NOT python
    # ... in colab, we can execute terminal commands by prefixing them with an exclamation point
    !wget -q $url 

# this should say True if the file got downloaded properly:
print(os.path.isfile(filepath))

DOWNLOADING all.csv
True


In [ ]:
from pandas import read_csv
country_codes_df = read_csv("all.csv")
country_codes = country_codes_df.to_dict("records")

##Currency Conversion

In [ ]:
import urllib.request, json 

def conversion(lowest_currency,lowest_price):
    with urllib.request.urlopen(f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency={lowest_currency}&to_currency=USD&apikey={API_KEY}') as url:
        data3 = json.load(url)
        if "Realtime Currency Exchange Rate" in data3:
            rt = float(data3["Realtime Currency Exchange Rate"]["5. Exchange Rate"])
            converted_price = rt * lowest_price
            print("-----------")
            print(rt)
            print(lowest_price)
            print(converted_price)
            return converted_price
        else:
            # print("Exchange Rate does not exist")
            return None



# Usage


In [ ]:
import requests
import json

all_codes=[]
app_list=[]

for c in country_codes:
    all_codes.append(str(c["alpha-2"]))

#print(all_codes)

request_url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"  
response = requests.get(request_url)
data = json.loads(response.text)

for d in data["applist"]["apps"]:
    app_list.append(str(d["appid"]))

#print(app_list)

app_id=input("Please enter the app id for the game you would like to purchase: ") 

while app_id not in app_list: 
        print ("Oops! Invalid app id. Please reenter: ")
        app_id=input()


Please enter the app id for the game you would like to purchase: 998940


In [ ]:
print(app_id)
# print(code)

861884


In [ ]:
lowest_price=1000000
lowest_currency="none"
location="none"

for code in all_codes:
        request_url=f"http://store.steampowered.com/api/appdetails?appids={app_id}&cc={code}&filters=price_overview"
        response = requests.get(request_url)
        data2 = json.loads(response.text)
        if data2 is not None:
            if data2[app_id]["success"]==False:
                lowest_price=lowest_price
                lowest_currency=lowest_currency
            else:
                if len(data2[app_id]["data"])==0:
                    print("This game is yet to be released. Please check later.")
                    break
                else:
                    price = data2[app_id]["data"]["price_overview"]["final"]/100
                    currency = data2[app_id]["data"]["price_overview"]["currency"]
                    price_in_usd=conversion(currency,price)
                    if price_in_usd is None:
                        continue
                    elif price_in_usd < lowest_price:
                        lowest_price = price_in_usd
                        lowest_currency = currency

if lowest_price != 1000000:
    print("Your selected game is cheapest in",lowest_currency,".")
    print("The lowest price in US dollar value is $%.2f."%(lowest_price))


-----------
0.14334656
28.0
4.013703680000001
-----------
0.1286455
38.0
4.888529
-----------
1.0
6.99
6.99
-----------
0.03271823
138.0
4.51511574
Your selected game is cheapest in CNY .
The lowest price in US dollar value is $4.01.
